In [1]:
using Flux, Statistics, BSON, Zygote
using Parameters: @with_kw

## ! This is Work in Progress.. Focus is 1st on Code materials - lecture notes (markdown) to be added immediately after, only placeholders for now !!

## Zygote gradients 

In [2]:
f = map(x -> 3x^2 + 2x + 1, 2)

17

In [3]:
df = gradient(x -> 3x^2 + 2x + 1, 1)

(8,)

In [4]:
g(x) = 5x^3 + 3x^2 + 2x

g (generic function with 1 method)

In [5]:
g(1.0)

10.0

In [6]:
typeof(g(1))

Int64

In [7]:
typeof(g(1.0))

Float64

In [8]:
x = 1;
dg = gradient(x -> g(x),x)

(23,)

In [9]:
g'(1.0)

23.0

In [10]:
g''(1.0)

36.0

In [11]:
@code_llvm g'(1.0)

;  @ /home/arvind/.julia/packages/Zygote/TaBlo/src/compiler/interface.jl:79 within `#48'
define double @"julia_#48_6790"(double %0) {
top:
; ┌ @ /home/arvind/.julia/packages/Zygote/TaBlo/src/compiler/interface.jl:76 within `gradient'
; │┌ @ /home/arvind/.julia/packages/Zygote/TaBlo/src/compiler/interface.jl:41 within `#46'
; ││┌ @ In[4]:1 within `Pullback'
; │││┌ @ /home/arvind/.julia/packages/ZygoteRules/OjfTt/src/adjoint.jl:59 within `#1817#back'
; ││││┌ @ /home/arvind/.julia/packages/Zygote/TaBlo/src/lib/number.jl:6 within `#245'
; │││││┌ @ promotion.jl:322 within `*' @ float.jl:332
        %1 = fmul double %0, 2.000000e+00
; │││││└
; │││││┌ @ float.jl:332 within `*'
        %2 = fmul double %1, 3.000000e+00
; │││││└
; │││││┌ @ intfuncs.jl:313 within `literal_pow'
; ││││││┌ @ float.jl:332 within `*'
         %3 = fmul double %0, %0
; │││││└└
; │││││┌ @ promotion.jl:322 within `*' @ float.jl:332
        %4 = fmul double %3, 3.000000e+00
; │││││└
; │││││┌ @ float.jl:332 within `*'
   

In [12]:
out = map((a,b) -> a^2 + b^2, 2,3)

13

In [13]:
a = (2,3)
typeof(a)

Tuple{Int64, Int64}

In [14]:
b = [2,3]
typeof(b)

Vector{Int64} (alias for Array{Int64, 1})

In [15]:
out_diff = gradient((a,b) -> a^2 + b^2, 2,3)

(4, 6)

In [16]:
# define as implicit function
abfunc(a,b) = a^2 + b^2;
out_diff2 = gradient((a,b) -> abfunc(a,b), 2,3)

(4, 6)

In [17]:
# define as explicit function
# if a,b are arrays
ainp = rand(3)
binp = rand(3)
cinp = rand(3)

function abfuncE(a,b)
    temp = a.^2 + b.^2
    out = (cinp - temp).^2
    return out
end

out_diff2 = gradient((a,b) -> sum(abfuncE(a,b)), ainp,binp)

([-0.01682912448487254, 0.35092026876742904, 0.5844391038001022], [-0.018791179106065214, 0.6608711873890442, 0.527195457877724])

In [18]:
# define as explicit function
# if a,b are arrays
ainp = rand(3,4)
binp = rand(4,3)
cinp = ones(3,3)

function abfuncE(a,b)
    temp = a * b
    out = (cinp - temp).^2
    return out
end

out_diff3 = gradient((a,b) -> sum(abfuncE(a,b)), ainp,binp)

([-1.8388029126611625 -1.8909131193676196 -1.270903504797593 -1.4087991139392744; 0.04240264844469612 -0.3007552618725052 0.027358374878052587 -0.18348267583440467; -1.1043498435027839 -0.6884639467539778 -0.918668035910722 -0.11885404542698645], [-0.7666143948576043 -0.5404682034873397 -0.4250911507581612; -0.49188630051413057 -0.11579571337120631 -0.4599633439177126; -1.006317074576978 -1.1208989239148213 -0.35184451879079354; -0.8370866894525162 -0.11493091647132882 -0.8288961312326126])

In [19]:
size(out_diff3[1])

(3, 4)

In [20]:
size(out_diff3[2])

(4, 3)

# ML
consider y = w * x + b, which is standard gradient descent problem 

L = RMS(yhat - y)

We need to compute dL/dw 

## Approach 1: Using parameters as tuples

In [40]:
# setting constant random numbers
using Random
Random.seed!(1234)
# define problem
Wo = rand(4,3);
bo = rand(4,1);
x = rand(3);
# Training data for Loss function definition
yhat = rand(4);

#Loss function
function Loss(W,b)
    y = W*x .+ b;
    L = 0.5*(yhat - y).^2;
    return L
end

L = Loss(Wo,bo);
print("Loss is ", L)

# Computing gradient w.r.t Loss
dLdw = gradient((W,b) -> sum(Loss(W,b)), Wo,bo)

Loss is [0.00334443227201436; 0.32591333255804367; 1.6725071823602882e-5; 0.13060259304603183]

([-0.022574505611653167 -0.053296669141485245 -0.004632531415641356; 0.22284767567185307 0.5261261993308384 0.04573074051817141; 0.0015963975069865641 0.003768971565172808 0.00032759794301538307; -0.141069440871712 -0.3330540852346647 -0.028948966939405282], [-0.08178547880906928; 0.8073578296617228; 0.005783609914854715; -0.5110823672286725])

In [41]:
Wo*x .+ bo

4×1 Matrix{Float64}:
 0.7609281377774828
 1.7578562368170239
 0.9704533862969444
 0.4346930380232398

## Approach 2: Using parameters as struct

In [42]:
struct Weights
    W
    b
end

(l::Weights)(x) = 0.5*(yhat - (l.W*x .+ l.b)).^2;

model = Weights(Wo, bo);

dLdw2 = gradient(model -> sum(model(x)), model)

((W = [-0.022574505611653167 -0.053296669141485245 -0.004632531415641356; 0.22284767567185307 0.5261261993308384 0.04573074051817141; 0.0015963975069865641 0.003768971565172808 0.00032759794301538307; -0.141069440871712 -0.3330540852346647 -0.028948966939405282], b = [-0.08178547880906928; 0.8073578296617228; 0.005783609914854715; -0.5110823672286725]),)

## Approach 2: Using parameters as dict

In [43]:
linearfunc(θ,x) = 0.5*(yhat - (θ[:W]*x .+ θ[:b])).^2;

θ = Dict(:W => Wo, :b => bo)

dθ = gradient(θ -> sum(linearfunc(θ,x)), θ)
print(dθ[1][:W])
print(dθ[1][:b])

[-0.022574505611653167 -0.053296669141485245 -0.004632531415641356; 0.22284767567185307 0.5261261993308384 0.04573074051817141; 0.0015963975069865641 0.003768971565172808 0.00032759794301538307; -0.141069440871712 -0.3330540852346647 -0.028948966939405282][-0.08178547880906928; 0.8073578296617228; 0.005783609914854715; -0.5110823672286725]

## Zygote Gradients _Using Flux layers_
same linear layer as above, but now with flux pre-defined functions

In [44]:
linearlayer = Dense(3,4);

In [45]:
linearlayer

Dense(3, 4)         # 16 parameters

In [46]:
out = linearlayer(x)

4-element Vector{Float64}:
  0.06093493508794132
 -0.39143472265048
  0.07506218364123313
 -0.40533673770076334

In [47]:
weights = Flux.params(linearlayer)

Params([Float32[-0.03939439 0.18283667 -0.8357598; 0.23633786 -0.6694229 -0.3606828; -0.50153035 0.40500325 -0.89034; -0.6558338 -0.38946676 0.5206107], Float32[0.0, 0.0, 0.0, 0.0]])

In [48]:
length(weights)

2

In [49]:
weights[1]

4×3 Matrix{Float32}:
 -0.0393944   0.182837  -0.83576
  0.236338   -0.669423  -0.360683
 -0.50153     0.405003  -0.89034
 -0.655834   -0.389467   0.520611

In [50]:
# backprop thru this layer
function loss(x, yhat)
  y = linearlayer(x)
  sum((yhat .- y).^2)
end

l = loss(x,yhat)
println("loss is: ", l)
gs = gradient(() -> loss(x,yhat), params(linearlayer))

loss is: 5.028868123612399


Grads(...)

In [51]:
typeof(gs)

Zygote.Grads

In [52]:
gs.params

Params([Float32[-0.03939439 0.18283667 -0.8357598; 0.23633786 -0.6694229 -0.3606828; -0.50153035 0.40500325 -0.89034; -0.6558338 -0.38946676 0.5206107], Float32[0.0, 0.0, 0.0, 0.0]])

In [53]:
keys(gs.grads)

KeySet for a IdDict{Any, Any} with 4 entries. Keys:
  :(Main.yhat)
  Float32[-0.03939439 0.18283667 -0.8357598; 0.23633786 -0.6694229 -0.3606828; …
  :(Main.x)
  Float32[0.0, 0.0, 0.0, 0.0]

In [54]:
#another way to computing gradients
ps = params(linearlayer)
gs2 = gradient(ps) do 
       loss(x,yhat)
end
typeof(gs)
gs.params

Params([Float32[-0.03939439 0.18283667 -0.8357598; 0.23633786 -0.6694229 -0.3606828; -0.50153035 0.40500325 -0.89034; -0.6558338 -0.38946676 0.5206107], Float32[0.0, 0.0, 0.0, 0.0]])

In [55]:
for p in params(linearlayer)
    println(gs[p])
end

[-0.4315745897572264 -1.0189143680865171 -0.08856374884289087; -0.7408033164416622 -1.7489795760986728 -0.1520208103452676; -0.49110066693821863 -1.1594508518254196 -0.10077914028200573; -0.7458705163116477 -1.76094284473411 -0.15306065427323046]
[-1.5635573629972215, -2.683866259611562, -1.7792151854817133, -2.702224285905351]


In [56]:
opt = ADAM()
α = 0.1 #learning rate
for p in params(linearlayer)
    println("before: ", p)
    println("loss before: ", loss(x,yhat))
    Flux.Optimise.update!(p,α * gs[p])
    println("after: ", p)
    println("loss after: ", loss(x,yhat))
end

before: Float32[-0.03939439 0.18283667 -0.8357598; 0.23633786 -0.6694229 -0.3606828; -0.50153035 0.40500325 -0.89034; -0.6558338 -0.38946676 0.5206107]
loss before: 5.028868123612399
after: Float32[0.0037630692 0.2847281 -0.82690346; 0.3104182 -0.49452496 -0.3454807; -0.4524203 0.52094835 -0.8802621; -0.58124673 -0.21337248 0.53591675]
loss after: 4.066032355256153
before: Float32[0.0, 0.0, 0.0, 0.0]
loss before: 4.066032355256153
after: Float32[0.15635574, 0.26838663, 0.17792152, 0.27022243]
loss after: 2.458428801436127


In [57]:
# Easier way of updating
opt = Descent(0.1) # Gradient descent with learning rate 0.1

for p in params(linearlayer)
    println("before: ", p) 
    println("loss before: ", loss(x,yhat))
    Flux.Optimise.update!(opt, p, gs[p])
    println("after: ", p)
    println("loss after: ", loss(x,yhat))
end

before: Float32[0.0037630692 0.2847281 -0.82690346; 0.3104182 -0.49452496 -0.3454807; -0.4524203 0.52094835 -0.8802621; -0.58124673 -0.21337248 0.53591675]
loss before: 2.458428801436127
after: Float32[0.046920527 0.38661954 -0.8180471; 0.3844985 -0.319627 -0.33027864; -0.40331024 0.63689345 -0.8701842; -0.5066597 -0.037278198 0.5512228]
loss after: 1.8006002770938272
before: Float32[0.15635574, 0.26838663, 0.17792152, 0.27022243]
loss before: 1.8006002770938272
after: Float32[0.31271148, 0.53677326, 0.35584304, 0.54044485]
loss after: 0.7980951557392415


## Zygote Internals

In [58]:
y, back = Zygote.pullback(sin,0.5)

(0.479425538604203, Zygote.var"#46#47"{Zygote.ZBack{ChainRules.var"#sin_pullback#1077"{Float64}}}(Zygote.ZBack{ChainRules.var"#sin_pullback#1077"{Float64}}(ChainRules.var"#sin_pullback#1077"{Float64}(0.8775825618903728))))

In [59]:
y

0.479425538604203

In [60]:
back(1)

(0.8775825618903728,)

In [61]:
cos(0.5)

0.8775825618903728

## Trying backprop thru arbitrary layers
## 1. FFT

In [62]:
using Random
using FFTW
Random.seed!(1234)
# define problem
x = rand(3);
# Training data for Loss function definition
yhat = rand(4);
linearlayer = Dense(3,4,σ);
model = Chain(linearlayer,x -> real(fft(x)));
#model = Chain(linearlayer,x -> imag(fft(x)));
out = model(x)
println(out)

# backprop thru this layer
function loss(x)
  y = model(x)
  sum((y).^2)
end

l = loss(x)
println("loss is: ", l)
gs = gradient(() -> loss(x), params(model))

[2.5681775741557797, 0.18202604677360879, 0.03501885108907654, 0.18202604677360879]
loss is: 6.66302933573632


Grads(...)

## 2. SVD

In [63]:
using Random
using LinearAlgebra
Random.seed!(1234)
# define problem
x = rand(3);
xmat = Float32.(rand(10,10,1,1)); #conv input (w,l,nch,batch)
yhatmat = Float32.(rand(10,10,1,1));
# Training data for Loss function definition
yhat = rand(4);
linearlayer = Dense(3,4,σ);
convlayer = Conv((3,3), 1 => 1, relu);
#model = Chain(linearlayer,x -> svd(x));
#model  = Chain(convlayer);
model = Chain(convlayer,
            x -> dropdims(x; dims=4),
            x -> dropdims(x; dims=3),
            x -> svd(x).S,
            x -> x.^3 + 2*x);
out = model(xmat);
@show size(out)

# backprop thru this layer
function loss(x)
  y = model(x)
  sum((y).^2)
end

l = loss(xmat)
#println("loss is: ", l)
gs = gradient(() -> loss(xmat), params(model))

size(out) = (8,)


Grads(...)

So adding SVD works.. although we have to drop dimensions and then add dimensions
every iteration for it to work (since SVD only accepts a m x n matrix that can't have channel and nbatches)

## 3. NNs with Arbitrary equations embedded
Lets consider input X matrix.
Define output Y which satisfies Y = X' + 2X.^2 
Problem: We have Y, but need to learn X such that the equation is satisfied.

This works, as amply demonstrated in documentation and my own experiments. Just find a good application problem to use it with.

In [64]:
using Random
using LinearAlgebra
Random.seed!(1234)
# define problem
x = rand(3);
dx = 0.1;
xmat = Float32.(rand(10,10,1,1)); #conv input (w,l,nch,batch)
yhatmat = Float32.(rand(10,10,1,1));
# Training data for Loss function definition
yhat = rand(4);
linearlayer = Dense(3,4,σ);
convlayer = Conv((3,3), 1 => 1, pad=1, relu);

function fdmlayer(x)
    fdmgradx = (x[2:end,:] - x[1:end-1,:])/(2*dx);
    fdmgrady = (x[:,2:end] - x[:,1:end-1])/(2*dx);
    gradfull = fdmgradx' * fdmgrady'
    #res = gradfull + 2 * x.^2
    return gradfull
end

model = Chain(convlayer,
            x -> dropdims(x, dims=4),
            x -> dropdims(x, dims=3),
            #x -> fdmlayer(x),
            x -> x.^3 + 2*x);
out = model(xmat);
@show size(out)

# backprop thru this layer
function loss(x)
  y = model(x)
  sum((y).^2)
end

l = loss(xmat)
#println("loss is: ", l)
gs = gradient(() -> loss(xmat), params(model))

size(out) = (10, 10)


Grads(...)

In [65]:
fdmgradx = (xmat[2:end,:] - xmat[1:end-1,:])/(2*dx);
fdmgrady = (xmat[:,2:end] - xmat[:,1:end-1])/(2*dx);

In [66]:
out = fdmgradx' * fdmgrady'
size(out)

(10, 10)

In [67]:
out2 = map((x) -> x.^3 + 2*x, out);

In [68]:
sum((out2).^2)

1.1367400223666475e10

## Avoiding Mutating Arrays with Zygote.Buffer
(In Progress)


In [82]:
x = rand(10);

function bufmodel(x)
    buf = Zygote.Buffer(x,length(x))
    for i=1:length(x)
        buf[i] = x[i]
    end
    return copy(buf)
end

function bufmodel2(x)
    buf = Zygote.Buffer(x,length(x));
    buf = Wavelets.dwt(x,wavelet(WT.coif6));
    return copy(buf)
end

gs = gradient(x -> sum(bufmodel2(x)), x);

LoadError: Mutating arrays is not supported -- called setindex!(::Vector{Float64}, _...)